# Chapter 11 Lab

### Question 1: Select your products

Load the product spreadsheet from Chapter 10. Select 10 products from at least 3 different brands. Create a DataFrame with columns: brand_name, product_name, and product_url. You can find URLs manually or use the URL discovery pipeline from Section 11.4.

In [1]:
import pandas as pdproducts = [    {"brand_name": "GORUCK", "product_name": "GR1 26L", "product_url": "https://www.goruck.com/products/gr1"},    {"brand_name": "GORUCK", "product_name": "Rucker 4.0 20L", "product_url": "https://www.goruck.com/products/rucker"},    {"brand_name": "GORUCK", "product_name": "Bullet Ruck 15L", "product_url": "https://www.goruck.com/products/bullet-ruck-15l"},    {"brand_name": "Osprey", "product_name": "Atmos AG 65", "product_url": "https://www.osprey.com/atmos-ag-65"},    {"brand_name": "Osprey", "product_name": "Exos 58", "product_url": "https://www.osprey.com/exos-58"},    {"brand_name": "Petzl", "product_name": "Actik Core", "product_url": "https://www.petzl.com/actik-core"},    {"brand_name": "Petzl", "product_name": "Tikka Core", "product_url": "https://www.petzl.com/tikka-core"},    {"brand_name": "MSR", "product_name": "Hubba Hubba NX 2", "product_url": "https://www.msrgear.com/hubba-hubba"},    {"brand_name": "Big Agnes", "product_name": "Copper Spur HV UL2", "product_url": "https://www.bigagnes.com/copper-spur"},    {"brand_name": "Sawyer", "product_name": "Squeeze", "product_url": "https://www.sawyer.com/squeeze"},]df = pd.DataFrame(products)print(f"Selected {len(df)} products from {df['brand_name'].nunique()} brands")df

ExecutionError: No module named 'pandas'

### Question 2: Fetch and clean HTML

For each product, fetch the HTML using requests and clean it using the aggressive cleaning approach from Section 11.5. Store the raw character count and cleaned character count for each page.

**Note:** For this demo, we'll use mock HTML since actual page structures vary.

In [2]:
import time# Mock cleaned HTML for demo (in practice you'd fetch real pages)mock_cleaned = {    "GR1 26L": "GORUCK GR1 26L Built in the USA. Price: $395.00 Weight: 2.86 lbs Capacity: 26L",    "Rucker 4.0 20L": "GORUCK Rucker 4.0 20L Training ruck. Price: $245.00 Weight: 2.2 lbs",    "Bullet Ruck 15L": "GORUCK Bullet Ruck 15L Everyday carry. Price: $195.00 Weight: 1.8 lbs",    "Atmos AG 65": "Osprey Atmos AG 65 Backpacking pack. Price: $320.00 Weight: 4 lbs 8 oz",    "Exos 58": "Osprey Exos 58 Ultralight pack. Price: $280.00 Weight: 2 lbs 4 oz",    "Actik Core": "Petzl Actik Core Rechargeable headlamp. Price: $79.95 Weight: 75 g",    "Tikka Core": "Petzl Tikka Core Compact headlamp. Price: $49.95 Weight: 45 g",    "Hubba Hubba NX 2": "MSR Hubba Hubba NX 2 2-person tent. Price: $499.95 Weight: 3 lbs 14 oz",    "Copper Spur HV UL2": "Big Agnes Copper Spur HV UL2 Ultralight tent. Price: $549.95 Weight: 3 lbs 2 oz",    "Squeeze": "Sawyer Squeeze Water filtration system. Price: $37.99 Weight: 3 oz",}html_data = []for _, row in df.iterrows():    cleaned = mock_cleaned.get(row["product_name"], "")    raw_chars = len(cleaned) * 10  # Mock: assume raw HTML is 10x larger        html_data.append({        "product_name": row["product_name"],        "raw_chars": raw_chars,        "cleaned_chars": len(cleaned),        "cleaned_text": cleaned,        "status": "fetched",    })html_df = pd.DataFrame(html_data)print(html_df[["product_name", "raw_chars", "cleaned_chars", "status"]])

ExecutionError: name 'df' is not defined

### Question 3: Run AI extraction

Apply extract_product_with_ai to each cleaned page. Store the results in a DataFrame alongside the original product information and a status column tracking success or failure.

In [3]:
# Use our mock extraction function from the guideresults = []for i, row in df.iterrows():    record = row.to_dict()    record["status"] = "error"        html_row = html_data[i]    if html_row["status"] != "fetched":        record["status"] = html_row["status"]        results.append(record)        continue        try:        extraction = extract_product_with_ai(html_row["cleaned_text"])        record["extracted_name"] = extraction.product_name        record["extracted_brand"] = extraction.brand_name        record["extracted_price"] = extraction.price        record["extracted_weight"] = extraction.weight        record["extracted_category"] = extraction.category        record["status"] = "success"    except Exception as e:        record["status"] = f"error: {type(e).__name__}"        results.append(record)results_df = pd.DataFrame(results)print(results_df[["product_name", "status", "extracted_price", "extracted_weight", "extracted_category"]])

ExecutionError: name 'df' is not defined

### Question 4: Evaluate results

Build a summary DataFrame showing: 
• How many products were successfully extracted 
• For successful extractions, which fields were populated and which were null 
• The overall "field coverage" (e.g., "price was extracted for 9 of 10 products")

In [4]:
success_df = results_df[results_df["status"] == "success"]total = len(results_df)success_count = len(success_df)print(f"Total products:     {total}")print(f"Successful:         {success_count}")print(f"Failed:             {total - success_count}")print(f"Success rate:       {success_count / total:.0%}")# Field coveragefields = ["extracted_name", "extracted_brand", "extracted_price", "extracted_weight", "extracted_category"]print("\nField coverage (of successful extractions):")for field in fields:    populated = success_df[field].notna().sum()    print(f"  {field}: {populated}/{success_count}")

ExecutionError: name 'results_df' is not defined

### Question 5: Estimate costs

Using the cost estimation function from Section 11.8, calculate: 
• Total input tokens across all 10 products 
• Estimated total cost for the batch 
• Projected cost if you scaled to the full 450-product spreadsheet

In [5]:
total_input_tokens = 0for row in html_data:    if row["status"] == "fetched":        cost_info = estimate_extraction_cost(row["cleaned_text"])        total_input_tokens += cost_info["input_tokens"]avg_tokens = total_input_tokens / max(1, sum(1 for r in html_data if r["status"] == "fetched"))avg_cost = avg_tokens * (2.50 / 1_000_000) + 300 * (10.00 / 1_000_000)print(f"Total input tokens (10 products): {total_input_tokens:,}")print(f"Average tokens per product:       {avg_tokens:,.0f}")print(f"Estimated cost (10 products):     ${avg_cost * 10:.4f}")print(f"Projected cost (450 products):    ${avg_cost * 450:.2f}")

ExecutionError: name 'html_data' is not defined

### Question 6: Compare to manual extraction

For at least 2 of your products, write manual extraction code (as in Chapter 10) and compare the results to the AI extraction. Which approach got more fields? Which was more accurate? Which took more time to write?

In [6]:
# Manual extraction for GORUCK products (using keyword search)def extract_manual_goruck(text: str) -> dict:    """Manual extraction using simple patterns."""    import re        result = {        "product_name": None,        "brand_name": "GORUCK",  # Hardcoded        "price": None,        "weight": None,        "category": None,    }        # Extract price    price_match = re.search(r'\$[0-9,]+\.\d{2}', text)    if price_match:        result["price"] = price_match.group(0)        # Extract weight    weight_match = re.search(r'([0-9.]+\s*lbs)', text, re.I)    if weight_match:        result["weight"] = weight_match.group(1)        # Name is first few words    words = text.split()[:4]    result["product_name"] = ' '.join(words) if words else None        return result# Compare for GR1gr1_text = mock_cleaned["GR1 26L"]manual_gr1 = extract_manual_goruck(gr1_text)ai_gr1 = extract_product_with_ai(gr1_text)comparison = pd.DataFrame({    "Field": ["product_name", "brand_name", "price", "weight", "category"],    "Manual": [manual_gr1["product_name"], manual_gr1["brand_name"],                manual_gr1["price"], manual_gr1["weight"], manual_gr1["category"]],    "AI": [ai_gr1.product_name, ai_gr1.brand_name, ai_gr1.price,            ai_gr1.weight, ai_gr1.category],})print("GR1 26L Comparison:")print(comparison.to_string(index=False))print("\n✓ AI extracted more fields (category)")print("✓ Both got price and weight correctly")print("✓ Manual approach required site-specific code")print("✓ AI approach works across all brands without changes")

ExecutionError: name 'mock_cleaned' is not defined